## What are the main drivers of the level of damage ? 

The goal of this part is to determine which variable explains the level of damage as an output of the damage function. To do so, we will proceed to an econometric regression of the level of damage by other explanatory variables. 

### Preparation of the data

In [2]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import numpy as np

xr.set_options(display_expand_attrs=False, display_expand_data=False)
np.set_printoptions(threshold=10, edgeitems=2)

In [4]:
# This cell stores the variables for the rest of the script, so it is easier to change if needed
data_file_path = '../../../results/batch/run_ds.nc'

First, we have to load the data that came as an output of the model. Variables of interest are stored in a netcdf file (.nc), that we will load as a dataset, and convert to a pandas dataframe. 

In [22]:
variables = ['define_tot_eq_define_total_impact',
       'dice_tot_eq_dice_total_impact', 'dsk_tot_eq_dsk_total_impact',
       'fund_tot_eq_moneraty_impact', 'fund_tot_eq_total_deaths',
       'fund_tot_eq_fund_total_impact', 'fund_tot_total_damage',
       'witness_tot_eq_witness_total_impact', 'time',
       'temperature_change', 'total_population', 'extra_extra_exponent',
       'extra_extra_normalisation_constant', 'total_radiative_forcing']

damage_variables = ['define_tot_eq_define_total_impact',
       'dice_tot_eq_dice_total_impact', 'dsk_tot_eq_dsk_total_impact',
       'fund_tot_eq_moneraty_impact', 'fund_tot_total_damage',
       'fund_tot_eq_fund_total_impact', 'witness_tot_eq_witness_total_impact']      

ds = xr.open_dataset(data_file_path)
df = ds[variables].to_dataframe().reset_index()

We now have a dataframe : 

In [14]:
df.head()

,time,REGIONS 35 I,define_tot_eq_define_total_impact,dice_tot_eq_dice_total_impact,dsk_tot_eq_dsk_total_impact,fund_tot_eq_moneraty_impact,fund_tot_eq_total_deaths,fund_tot_eq_fund_total_impact,fund_tot_total_damage,witness_tot_eq_witness_total_impact,temperature_change,total_population,extra_extra_exponent,extra_extra_normalisation_constant,total_radiative_forcing
0,2005.0,AUSTRIA,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,0.000000,0.845,6.541007e+09,1.726997,40251.56495,1.714491
1,2005.0,BELGIUM,405.802677,490.210341,0.0,1.379671e+09,5.746519e+06,2.846619e+13,4.973581e+06,828.299372,0.845,6.541007e+09,1.726997,40251.56495,1.714491
2,2005.0,BULGARIA,2.116532,2.556774,0.0,2.799410e+08,5.009851e+01,3.229255e+08,8.615126e+05,4.320134,0.845,6.541007e+09,1.726997,40251.56495,1.714491
3,2005.0,CROATIA,5.642319,6.815932,0.0,2.565073e+08,2.853332e+01,2.996817e+08,1.519235e+06,11.516754,0.845,6.541007e+09,1.726997,40251.56495,1.714491
4,2005.0,CYPRUS,7.735754,9.344804,0.0,9.993271e+07,5.360278e+05,1.631465e+12,3.055627e+06,15.789743,0.845,6.541007e+09,1.726997,40251.56495,1.714491


We need to have all the damages in the same column to process to the regression : 

In [23]:
df_melted = df.melt(id_vars=['time', 'REGIONS 35 I', 'extra_extra_exponent', 'extra_extra_normalisation_constant', 'total_radiative_forcing'], var_name='equation',  value_vars=damage_variables, value_name='total_damage')
df_melted


,time,REGIONS 35 I,extra_extra_exponent,extra_extra_normalisation_constant,total_radiative_forcing,equation,total_damage
0,2005.0,AUSTRIA,1.726997,40251.56495,1.714491,define_tot_eq_define_total_impact,0.000000e+00
1,2005.0,BELGIUM,1.726997,40251.56495,1.714491,define_tot_eq_define_total_impact,4.058027e+02
2,2005.0,BULGARIA,1.726997,40251.56495,1.714491,define_tot_eq_define_total_impact,2.116532e+00
3,2005.0,CROATIA,1.726997,40251.56495,1.714491,define_tot_eq_define_total_impact,5.642319e+00
4,2005.0,CYPRUS,1.726997,40251.56495,1.714491,define_tot_eq_define_total_impact,7.735754e+00
...,...,...,...,...,...,...,...
11265,2050.0,INDIA,1.726997,40251.56495,3.804644,witness_tot_eq_witness_total_impact,5.258178e+01
11266,2050.0,LATAM,1.726997,40251.56495,3.804644,witness_tot_eq_witness_total_impact,1.601187e+04
11267,2050.0,RUSSIA,1.726997,40251.56495,3.804644,witness_tot_eq_witness_total_impact,1.402535e+05
11268,2050.0,USMCA,1.726997,40251.56495,3.804644,witness_tot_eq_witness_total_impact,3.077925e+06


In [48]:
import numpy as np
import pandas as pd

initial_time = 2005
final_time = 2055
time_step = 5 

for i in range(initial_time, final_time + time_step, time_step):
    print(i)

time = np.linspace(initial_time, final_time, num=(final_time - initial_time)//time_step + 1)
print(time)

random = np.random.normal(0, 1, len(time))

init_rand = pd.Series(random, index=time)
init_rand

2005
2010
2015
2020
2025
2030
2035
2040
2045
2050
2055
[2005. 2010. ... 2050. 2055.]


2005.0   -0.571384
2010.0    1.223343
2015.0   -1.421513
2020.0    1.053245
2025.0    0.722742
2030.0   -0.647791
2035.0    3.048792
2040.0   -1.070241
2045.0   -2.261687
2050.0   -1.394331
2055.0   -0.054942
dtype: float64

## Preparing the regression

### With all variables

In [27]:
# Group by time, region, and equation, and sum the total damage (aggregated data)
df_regression = df_melted.groupby(['time', 'REGIONS 35 I', 'extra_extra_exponent', 'extra_extra_normalisation_constant', 'total_radiative_forcing', 'equation']).sum().reset_index()

# Create a dummy for each damage function
df_regression = pd.get_dummies(df_regression, columns=['equation'], drop_first=True)

# Drop the null values
df_regression = df_regression.dropna()

# Define the dependant variable and the independant variables
y = df_regression['total_damage']
X = df_regression.drop(columns=['total_damage', 'REGIONS 35 I'], axis=1)

# Add a constant to the independant variables
X = sm.add_constant(X)

# Convert the data to float
X = X.astype(float)
y = y.astype(float)

# Fit the model
model = sm.OLS(y, X)
results = model.fit()

# Print the results
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           total_damage   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     217.3
Date:                Tue, 16 Jul 2024   Prob (F-statistic):               0.00
Time:                        10:58:25   Log-Likelihood:            -3.6498e+05
No. Observations:               11270   AIC:                         7.300e+05
Df Residuals:                   11261   BIC:                         7.300e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
time                                          4.036e+11   4.94e+11      0.818      0.414   -5.64e+11    1.37e+12
extra_extra_exponent                         -8.479e+05   1.04e+06     -0.818      0.413   -2.88e+06    1.18e+06
extra_extra_normalisation_constant           -1.976e+10   2.41e+10     -0.818      0.413   -6.71e+10    2.76e+10
total_radiative_forcing                      -8.311e+12   1.05e+13     -0.789      0.430    -2.9e+13    1.23e+13
equation_dice_tot_eq_dice_total_impact       -1.381e+04    9.9e+11   -1.4e-08      1.000   -1.94e+12    1.94e+12
equation_dsk_tot_eq_dsk_total_impact         -6.857e+04    9.9e+11  -6.93e-08      1.000   -1.94e+12    1.94e+12
equation_fund_tot_eq_fund_total_impact        3.152e+13    9.9e+11     31.834      0.000    2.96e+13    3.35e+13
equation_fund_tot_eq_moneraty_impact          1.407e+10    9.9e+11      0.014      0.989   -1.93e+12    1.95e+12
equation_fund_tot_total_damage                 4.49e+06    9.9e+11   4.54e-06      1.000   -1.94e+12    1.94e+12
equation_witness_tot_eq_witness_total_impact  2.549e+04    9.9e+11   2.58e-08      1.000   -1.94e+12    1.94e+12
==============================================================================
Omnibus:                    15420.903   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2917704.370
Skew:                           8.120   Prob(JB):                         0.00
Kurtosis:                      80.134   Cond. No.                     8.69e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.43e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""